# Install h2o

In [1]:
#!pip install h2o
#please note that h2o requires JAVA

# Load Packages and Initialize

In [2]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
import pandas as pd
# Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "19.0.2" 2023-01-17; Java(TM) SE Runtime Environment (build 19.0.2+7-44); Java HotSpot(TM) 64-Bit Server VM (build 19.0.2+7-44, mixed mode, sharing)
  Starting server from /opt/anaconda3/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/pf/bjt8ngz536v9z189p8lxdv1m0000gn/T/tmpkc_bgnz3
  JVM stdout: /var/folders/pf/bjt8ngz536v9z189p8lxdv1m0000gn/T/tmpkc_bgnz3/h2o_samueltchakwera_started_from_python.out
  JVM stderr: /var/folders/pf/bjt8ngz536v9z189p8lxdv1m0000gn/T/tmpkc_bgnz3/h2o_samueltchakwera_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_samueltchakwera_wgmb2u
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


# Load Data

In [3]:
# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")


# Process data 

In [4]:
# Remove unnecessary columns
train_data = train_data.drop(["PassengerId", "Name"], axis=1)
test_data = test_data.drop(["PassengerId", "Name"], axis=1)

# Convert data to H2OFrame
train_data = h2o.H2OFrame(train_data)
test_data = h2o.H2OFrame(test_data)

# Define the features and target
features = train_data.columns
target = "Transported"
features.remove(target)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


# Create, Train and Predict using Random forest

In [5]:
# Create and train the H2O Random Forest Classifier
rf_classifier = H2ORandomForestEstimator(seed=42)
rf_classifier.train(x=features, y=target, training_frame=train_data)

# Validate the model
val_data = train_data.split_frame(ratios=[0.8], seed=42)[1]
val_pred = rf_classifier.predict(val_data)
accuracy = (val_pred["predict"] == val_data[target]).as_data_frame().mean()[0]
print(f"Validation accuracy: {accuracy:.2f}")

# Predict the 'Transported' column for the test data
test_data["Transported"] = rf_classifier.predict(test_data)["predict"]

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Validation accuracy: 0.97
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/opt/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'Cabin' has levels not trained on: ["A/1/P", "A/105/S", "A/108/S", "A/109/S", "A/11/P", "A/12/P", "A/12/S", "A/13/S", "A/15/P", "A/15/S", ...3245 not listed..., "G/993/P", "G/994/P", "G/995/P", "G/996/P", "G/997/P", "G/997/S", "T/0/S", "T/1/S", "T/3/S", "T/4/P"]
  warnings.warn(w)


# Save submission file and shutdown h2o

In [6]:
# Create the submission file
submission = pd.read_csv("sample_submission.csv")
submission["Transported"] = test_data["Transported"].as_data_frame()
submission.to_csv("RF_submission.csv", index=False)

# Shutdown H2O
h2o.cluster().shutdown()

H2O session _sid_b5fb closed.
